In [ ]:
import pandas as pd
import os

In [ ]:
#males
#45880669
#45878463

In [ ]:
visit_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` visit 
        WHERE
        visit.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

visit_df = pd.read_gbq(visit_sql, dialect="standard")
visit_df=visit_df.sort_values(by=['person_id','visit_start_datetime'])
visit_df.head(5)

In [ ]:
visit_df=visit_df.sort_values(by=['person_id','visit_start_datetime'])

In [ ]:
visit_df.head(5)

In [ ]:
visit_df=visit_df[['visit_occurrence_id', 'person_id',
       'visit_start_datetime', 'visit_end_datetime',
       'preceding_visit_occurrence_id']]

In [ ]:
visit_df.to_csv('visitMale.csv',index=False)

In [ ]:
visit_df.shape

In [ ]:
len(list(visit_df['visit_occurrence_id'].unique()))

In [ ]:
len(list(visit_df['person_id'].unique()))

In [ ]:
visit_df = pd.read_csv('visitMale.csv',header=0)

In [ ]:
visit_df=visit_df.sort_values(by=['person_id','visit_start_datetime'])

In [ ]:
visit_df.to_csv('visitMale.csv',index=False)

In [ ]:
#Calculate age at each visit
from datetime import datetime
visit_df['visit_start_datetime'] = pd.to_datetime(visit_df['visit_start_datetime'])
visit_df['birth_datetime'] = pd.to_datetime(visit_df['birth_datetime'])

In [ ]:
visit_df['age']=visit_df['visit_start_datetime'] - visit_df['birth_datetime']

In [ ]:
visit_df[['age']].head()

In [ ]:
#Convert age to years
visit_df['age']=(((visit_df['age'].dt.total_seconds()/3600)/24)/30)

In [ ]:
visit_df[['age']].head()

In [ ]:
visit_df['age']=visit_df['age']/12

In [ ]:
#############MEAS##############

In [ ]:
meas_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.measurement` meas 
        WHERE
        (
            meas.measurement_concept_id IN (
                40762636, 3004249, 3027018, 3012888, 3038553, 3036277, 3025315, 3000483
            ) 
            OR  meas.measurement_source_concept_id IN (
                40762636, 3004249, 3027018, 3012888, 3038553, 3036277, 3025315, 3000483
            )
        )  and
        meas.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

meas_df = pd.read_gbq(meas_sql, dialect="standard")
meas_df.head(5)

In [ ]:
meas_df.shape

In [ ]:
meas_df = meas_df[meas_df['value_as_number'].notnull()]

In [ ]:
meas_df=meas_df.sort_values(by=['person_id','visit_occurrence_id'])

In [ ]:
meas_df.shape

In [ ]:
meas_df.to_csv('measM.csv',index=False)

In [ ]:
meas_df=pd.merge(meas_df,visit_df[['person_id','visit_occurrence_id','age']],on=['person_id','visit_occurrence_id'],how='left')

In [ ]:
meas_df.columns

In [ ]:
meas_df=meas_df[['measurement_id', 'person_id', 'measurement_concept_id',
        'value_as_number', 'visit_occurrence_id']]

In [ ]:
meas_df.to_csv('measM.csv',index=False)

In [ ]:
#####SELECT MEAS Cohort##############

In [ ]:
cond_df=pd.read_csv('condM.csv',header=0)

In [ ]:
cond_df.head()

In [ ]:
del cond_df['condition_concept_id']
del cond_df['concept_name']

In [ ]:
cond_df = cond_df.rename(columns={'ancestor_id3':'condition_concept_id'})
cond_df = cond_df.rename(columns={'ancestor_name3':'concept_name'})

In [ ]:
meas_df=pd.read_csv('measM.csv',header=0)

In [ ]:
meas_df.head()

In [ ]:
len(list(meas_df['person_id'].unique()))

In [ ]:
meas_df=meas_df[meas_df['person_id'].isin(list(cond_df['person_id'].unique()))]

In [ ]:
len(list(meas_df['person_id'].unique()))

In [ ]:
meas_df=meas_df[meas_df['visit_occurrence_id'].isin(list(cond_df['visit_occurrence_id'].unique()))]

In [ ]:
len(list(meas_df['person_id'].unique()))

In [ ]:
len(list(meas_df['visit_occurrence_id'].unique()))

In [ ]:
len(list(cond_df['visit_occurrence_id'].unique()))

In [ ]:
#####Done selecting cohort###################

In [ ]:
###############COND################

In [ ]:
cond_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` cond 
        WHERE
        cond.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

cond_df = pd.read_gbq(cond_sql, dialect="standard")
cond_df.head(5)

In [ ]:
cond_df.shape

In [ ]:
cond_df.to_csv('condM.csv',index=False)

In [ ]:
cond_df=pd.read_csv('condM.csv',header=0)

In [ ]:
cond_df.shape

In [ ]:
cond_df.head()

In [ ]:
len(list(cond_df['person_id'].unique()))

In [ ]:
##############Combine with visit data to get age####################

In [ ]:
visit=pd.read_csv('visitMale.csv',header=0)

In [ ]:
visit.head()

In [ ]:
cond_df=pd.merge(cond_df,visit[['visit_occurrence_id', 'person_id','age']],how='left',on=['person_id','visit_occurrence_id'])

In [ ]:
cond_df = cond_df[cond_df['age'].notna()]

In [ ]:
cond_df.shape

In [ ]:
len(list(cond_df['person_id'].unique()))

In [ ]:
cond_df.to_csv('condM.csv',index=False)

In [ ]:
len(list(cond_df['condition_concept_id'].unique()))

In [ ]:
#########Code Statistics##############

In [ ]:
ncond = (cond_df[['condition_concept_id','person_id','visit_occurrence_id']].groupby(['condition_concept_id']).count()).reset_index()

In [ ]:
ncond['condition_concept_id'].max()

In [ ]:
ncond['condition_concept_id'].mean()

In [ ]:
ncond['condition_concept_id'].min()

In [ ]:
######Number of condition codes above mean##################
ncond[ncond['condition_concept_id']>7760039].shape

In [ ]:
######Number of condition codes below mean##################
ncond[ncond['condition_concept_id']<=7760039].shape

In [ ]:
#####Population loevel nCond###########
ncond=cond_df[['condition_concept_id','person_id']].groupby(['condition_concept_id','person_id']).size().reset_index(name='counts')

In [ ]:
ncond.head()

In [ ]:
#####Population loevel nCond###########
ncond=ncond[['condition_concept_id','person_id']].groupby(['condition_concept_id']).size().reset_index(name='counts')

In [ ]:
ncond.head()

In [ ]:
ncond['counts'].max()

In [ ]:
ncond['counts'].mean()

In [ ]:
######Number of condition codes above mean##################
ncond[ncond['counts']>96].shape

In [ ]:

######Number of condition codes below mean##################
ncond[ncond['counts']<=96].shape

In [ ]:
acids = ncond[ncond['counts']>96]['condition_concept_id']
bcids = ncond[ncond['counts']<=96]['condition_concept_id']

In [ ]:
cond_dfa=cond_df[cond_df['condition_concept_id'].isin(acids)]

In [ ]:
cond_dfa=cond_dfa[['person_id', 'condition_concept_id','visit_occurrence_id','age','concept_name']]

In [ ]:
###########CONCEPT_RELATIONSHIP#################

In [ ]:
rel_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.concept_relationship` cond """

rel_df = pd.read_gbq(rel_sql, dialect="standard")
rel_df.head(5)

In [ ]:
rel_df.columns

In [ ]:
rel_df=rel_df[['concept_id_1', 'concept_id_2', 'relationship_id']]

In [ ]:
rel_df.to_csv('rel.csv',index=False)

In [ ]:
###### GROUP CONDS #######

In [ ]:
rel_df=pd.read_csv('rel.csv',header=0)

In [ ]:
rel_df.head()

In [ ]:
concept_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` cond 
    where
                    concept_id in (195083,194755,4109084,4182562)"""

concept_df = pd.read_gbq(concept_sql, dialect="standard")
concept_df

In [ ]:
concept_df.to_csv('concept.csv',index=False)

In [ ]:
#########Another Method#############

In [ ]:
rel_df=rel_df[rel_df['relationship_id']=='Is a']

In [ ]:
rel_df.columns

In [ ]:
rel_df=rel_df[['concept_id_1', 'concept_id_2']]

In [ ]:
concept=pd.read_csv('concept.csv',header=0)

In [ ]:
concept=concept[concept['concept_name'].isin(['General finding of observation of patient','Finding of body region','Disorder of body system','Inflammation of specific body organs','Finding by site','Complication','Disease','Clinical finding','Abdominal organ finding','Acute disease','Chronic disease','Choroidal and/or chorioretinal disorder','Evaluation finding','Disorder by body site','Measurement finding','Measurement finding below reference range','Measurement finding outside reference range','SNOMED CT Concept','Clinical history and observation findings','Degenerative disorder','Disease related state','Disorder in full remission','Finding by method','Functional finding','General body state finding'])]

In [ ]:
rel_df=rel_df[~rel_df['concept_id_2'].isin(list(concept['concept_id']))]

In [ ]:
concept.head()

In [ ]:
concept=pd.read_csv('concept.csv',header=0)

In [ ]:
concept=concept.rename(columns={'concept_id':'condition_concept_id'})

In [ ]:
cond_dfa=pd.merge(cond_dfa,concept,how='left',on='condition_concept_id')

In [ ]:
concept=concept.rename(columns={'condition_concept_id':'concept_id'})

In [ ]:
rel_df=rel_df[rel_df['concept_id_1']!=43530680]

In [ ]:
rel_df=rel_df.groupby('concept_id_1').first().reset_index()

In [ ]:
rel_df.head()

In [ ]:
rel_df=rel_df.rename(columns={'concept_id_2':'concept_id'})
rel_df=pd.merge(rel_df, concept, on='concept_id',how='left')

In [ ]:
len(list(cond_dfa['person_id'].unique()))

In [ ]:
rel_df.head()

In [ ]:
rel_df=rel_df.rename(columns={'concept_id_1':'condition_concept_id'})
rel_df=rel_df.rename(columns={'concept_id':'ancestor_id'})
rel_df=rel_df.rename(columns={'concept_name':'ancestor_name'})

In [ ]:
cond_dfa=cond_dfa.fillna(0)

In [ ]:
cond_dfa=cond_dfa[cond_dfa['condition_concept_id']!=0]

In [ ]:
cond_dfa=pd.merge(cond_dfa, rel_df, on='condition_concept_id',how='left')

In [ ]:
len(list(cond_dfa['condition_concept_id'].unique()))

In [ ]:
len(list(cond_dfa['ancestor_id'].unique()))

In [ ]:
cond_dfa=cond_dfa.fillna(0)

In [ ]:
cond_dfa.loc[cond_dfa['ancestor_id']==0,'ancestor_id']=cond_dfa.loc[cond_dfa['ancestor_id']==0,'condition_concept_id']
cond_dfa.loc[cond_dfa['ancestor_name']==0,'ancestor_name']=cond_dfa.loc[cond_dfa['ancestor_name']==0,'concept_name']

In [ ]:
cond_dfa=pd.read_csv('cond_dfa.csv',header=0)

In [ ]:
rel_df=rel_df.rename(columns={'ancestor_id':'ancestor_id2'})
rel_df=rel_df.rename(columns={'ancestor_name':'ancestor_name2'})
rel_df=rel_df.rename(columns={'condition_concept_id':'ancestor_id'})

In [ ]:
rel_df.head()

In [ ]:
cond_dfa=pd.merge(cond_dfa, rel_df, on=['ancestor_id'],how='left')

In [ ]:
len(list(cond_dfa['ancestor_id2'].unique()))

In [ ]:
cond_dfa=cond_dfa.fillna(0)

In [ ]:
cond_dfa.loc[cond_dfa['ancestor_id2']==0,'ancestor_id2']=cond_dfa.loc[cond_dfa['ancestor_id2']==0,'ancestor_id']
cond_dfa.loc[cond_dfa['ancestor_name2']==0,'ancestor_name2']=cond_dfa.loc[cond_dfa['ancestor_name2']==0,'ancestor_name']

In [ ]:
rel_df=rel_df.rename(columns={'ancestor_id2':'ancestor_id3'})
rel_df=rel_df.rename(columns={'ancestor_name2':'ancestor_name3'})
rel_df=rel_df.rename(columns={'ancestor_id':'ancestor_id2'})

In [ ]:
rel_df.head()

In [ ]:
cond_dfa=pd.merge(cond_dfa, rel_df, on=['ancestor_id2'],how='left')

In [ ]:
len(list(cond_dfa['ancestor_id3'].unique()))

In [ ]:
cond_dfa=cond_dfa.fillna(0)

In [ ]:
cond_dfa.loc[cond_dfa['ancestor_id3']==0,'ancestor_id3']=cond_dfa.loc[cond_dfa['ancestor_id3']==0,'ancestor_id2']
cond_dfa.loc[cond_dfa['ancestor_name3']==0,'ancestor_name3']=cond_dfa.loc[cond_dfa['ancestor_name3']==0,'ancestor_name2']

In [ ]:
len(list(cond_dfa['person_id'].unique()))

In [ ]:
cond_dfa.to_csv('cond_dfa.csv',index=False)

In [ ]:
cond_dfa=cond_dfa.sort_values(['person_id','age'])

In [ ]:
cond_dfa.columns

In [ ]:
cond_dfa=cond_dfa[['person_id', 'condition_concept_id', 'visit_occurrence_id', 'age',
       'concept_name','ancestor_id3', 'ancestor_name3']]

In [ ]:
cond_dfa=cond_dfa.fillna(0)

In [ ]:
################End of Grouping##############

In [ ]:
cond_dfa=pd.read_csv('cond_dfa.csv',header=0)

In [ ]:
#max number of visits
(cond_dfa.groupby(['person_id'])['visit_occurrence_id'].count()).mean()

In [ ]:
startVisit = cond_dfa.groupby(['person_id'])['age'].min()
endVisit = cond_dfa.groupby(['person_id'])['age'].max()

In [ ]:
startVisit.reset_index()

In [ ]:
endVisit.reset_index()

In [ ]:
medLen = (endVisit - startVisit).reset_index()

In [ ]:
medLen

In [ ]:
medLen['age'].max()

In [ ]:
medLen['age'].min()

In [ ]:
medLen['age'].mean()

In [ ]:
medLen=medLen[(medLen['age']>=2) & (medLen['age']<=10)]

In [ ]:
pids=list(medLen['person_id'].unique())

In [ ]:
cond_dfa=cond_dfa[cond_dfa['person_id'].isin(pids)]

In [ ]:
len(list(cond_dfa['person_id'].unique()))

In [ ]:
cond_dfa.to_csv('condF.csv',index=False)

In [ ]:
############end of cohort selection################

In [ ]:
drug_sql = """
    SELECT *
    from
        `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` drug 
        WHERE
        drug.PERSON_ID IN (
            select
                person_id  
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    where
                        sex_at_birth_concept_id in (45880669) 
                ) 
            )"""

drug_df = pd.read_gbq(drug_sql, dialect="standard")
drug_df.head(5)

In [ ]:
drug_df.shape

In [ ]:
drug_df=drug_df[['drug_exposure_id', 'person_id', 'drug_concept_id','visit_occurrence_id','drug_source_concept_id']]

In [ ]:
drug_df.to_csv('drugF.csv',index=False)

In [ ]:
drug_df=pd.read_csv('drugF.csv',header=0)

In [ ]:
cond_df=pd.read_csv('condF.csv',header=0)

In [ ]:
len(list(drug_df['person_id'].unique()))

In [ ]:
drug_df=drug_df[drug_df['person_id'].isin(list(cond_df['person_id'].unique()))]

In [ ]:
len(list(drug_df['person_id'].unique()))

In [ ]:
drug_df=drug_df[drug_df['visit_occurrence_id'].isin(list(cond_df['visit_occurrence_id'].unique()))]

In [ ]:
len(list(drug_df['person_id'].unique()))

In [ ]:
len(list(drug_df['person_id'].unique()))

In [ ]:
######Remove below mean ids################

In [ ]:
ncond=drug_df[['drug_concept_id','person_id']].groupby(['drug_concept_id','person_id']).size().reset_index(name='counts')

In [ ]:
del ncond['counts']

In [ ]:
ncond=ncond[['drug_concept_id','person_id']].groupby(['drug_concept_id']).size().reset_index(name='counts')

In [ ]:
ncond['counts'].max()

In [ ]:
ncond['counts'].mean()

In [ ]:
acids = ncond[ncond['counts']>37]['drug_concept_id']

In [ ]:
drug_df=drug_df[drug_df['drug_concept_id'].isin(acids)]

In [ ]:
###########grouping drug###############

In [ ]:
drug_df.head()

In [ ]:
len(list(drug_df['drug_concept_id'].unique()))

In [ ]:
rel_df=pd.read_csv('rel.csv',header=0)

In [ ]:
rel_df.columns

In [ ]:
rel_df=rel_df.groupby('concept_id_1').first().reset_index()

In [ ]:
rel_df.head()

In [ ]:
drug_df.head()

In [ ]:
rel_df=rel_df[rel_df['relationship_id']=='RxNorm - ATC']

In [ ]:
rel_df=rel_df[['concept_id_1','concept_id_2']]

In [ ]:
drug_df = drug_df[drug_df['drug_concept_id']!=0]

In [ ]:
drug_df.shape

In [ ]:
len(list(drug_df['person_id'].unique()))

In [ ]:
rel_df=rel_df.rename(columns={'concept_id_1':'drug_concept_id'})
rel_df=rel_df.rename(columns={'concept_id_2':'ancestor1'})

In [ ]:
drug_df=pd.merge(drug_df, rel_df, on='drug_concept_id',how='left')

In [ ]:
len(list(drug_df['ancestor1'].unique()))

In [ ]:
drug_df=drug_df.fillna(0)

In [ ]:
drug_df.loc[drug_df['ancestor1']==0,'ancestor1']=drug_df.loc[drug_df['ancestor1']==0,'drug_concept_id']

In [ ]:
len(list(drug_df['ancestor1'].unique()))

In [ ]:
rel_df=rel_df.rename(columns={'ancestor1':'ancestor2'})
rel_df=rel_df.rename(columns={'drug_concept_id':'ancestor1'})

In [ ]:
drug_df=pd.merge(drug_df, rel_df, on='ancestor1',how='left')

In [ ]:
(list(drug_df['ancestor2'].unique()))

In [ ]:
drug_df=drug_df.fillna(0)

In [ ]:
drug_df.loc[drug_df['ancestor2']==0,'ancestor2']=drug_df.loc[drug_df['ancestor2']==0,'ancestor1']

In [ ]:
len(list(drug_df['ancestor2'].unique()))

In [ ]:
drug_df.head()

In [ ]:
drug_df.columns

In [ ]:
drug_df=drug_df[[ 'person_id', 'drug_concept_id',
       'visit_occurrence_id','ancestor1' ]]

In [ ]:
drug_df.to_csv('drugFReduced.csv',index=False)

In [ ]:
len(list(drug_df['person_id'].unique()))

In [ ]:
####end of removing ids###########

In [ ]:
concept=pd.read_csv('concept.csv',header=0)

In [ ]:
concept.head()

In [ ]:
concept=concept.rename(columns={'concept_id':'ancestor1'})

In [ ]:
drug_df=pd.merge(drug_df,concept,how='left',on=['ancestor1'])

In [ ]:
len(list(drug_df['concept_name'].unique()))

In [ ]:
drug_df.head()

In [ ]:
drug_df.to_csv('drugM.csv',index=False)